<p></p><font size=8 face="courier" color="magenta">Visualize</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# Mock Setup

Trying to get javascript to ork, putting everythign in 1 file to see what happens.

In [3]:
JS("""
var cy;
require(['cytoscape'], function(cytoscape){ 
    cy = cytoscape({ container: document.getElementById('cy') });
    

cy.io = {}       // <-- where we store all the parameters/variables we create        
cy.maxZoom(14)   // <-- Determine how much window zooms in and out
cy.minZoom(.4)

cy.on("select", 'node, edge' , function(){     //<-- highlight node when selected
    this.style( 'overlay-opacity', .4 );
});
cy.on("unselect", 'node, edge' , function(){   //<-- unhighlight when not selected
    this.style( 'overlay-opacity', 0 );
});

// Create default style for nodes and edges
cy.io.style = {}
cy.io.style.node = { 'overlay-color': 'gray', 'overlay-padding': '2px', 'background-color': 'gray', 'text-background-color':'white',
                        'text-border-width':'2px', 'text-background-padding':'4px' }
cy.io.style.edge = { 'overlay-color': 'gray', 'overlay-padding': '8px', 'line-color': 'gray', 'curve-style': 'bezier', 
                        'text-background-color':'white', 'text-border-width':'2px', 'text-background-padding':'4px' }

// When selecting/unselecting elements, Save selection search (for computational efficiency)                      
cy.on('select unselect', 'node, edge', function(event){ cy.io.selection= cy.$(':selected')  }); 





cy.io.style.set = function(ele, self=false){
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;
    
    if( self == false ){ //<-- set properties of element
        for (var property in style ) {
            if(  !/[A-Z]/.test(property) && !property.includes("pie") && ele.style(property) != "NaNpx"  ){   //  remove cytoscape.js errors
                ele.style( property, style[property] )
           }
        }    
    }else{    
        for( var property in ele.style() ){ 
            if(  !/[A-Z]/.test(property) && !property.includes("pie") && ele.style(property) != "NaNpx"  ){   //  remove cytoscape.js errors
                style[ property ] = ele.style( property ) 
           }
        }
    }
};



cy.io.addNode = function(event){
  if( event.target === cy ){                                                          // <-- If we tapped on the background
      var ele = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      cy.io.style.set( ele )
  }; 
};
cy.on('tap', cy.io.addNode );




cy.io.click_hist = ["1", "2", "3", "4"];

cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
    cy.io.click_hist.shift();                                         //<-- Remove oldest click

    if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
      cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

      if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){           //<-- If 2 double clicks
          var ele= cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);    //<-- add edge
          cy.io.style.set( ele )                                                                            //<-- add style of new edge
      };

    }else{                                                 //<-- if we didn't click a node
      cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
    }; 
};
cy.on("click" , cy.io.addEdge );






cy.io.style.change = function(ele, property, value ){
    cy.io.style.set( ele, self=true )                                     // Make style same as node type
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;   // get style type
    
    typeof(property)
    if( typeof(property)=="string" ){                                     // handle changing a single style property
        style[ property ] = value                                             // change style value
        ele.style( property, value )                                          // change element's value
    }else{                                                                // Multiple properties
        for(var i=0; i<property.length; i++){
            style[ property[i] ] = value[i]
            ele.style( property[i], value[i] )  
        }
    }
};



});
""")

In [2]:
# My Canvas Box
display(HTML("""<style> .cy { width: 50vw; height: 30vw}</style>   //DEBUG: made width 50vw
                <div id="cy" class="cy"></div>"""))

# Setup Notebook

## Load Stuff

`%cd` sets the path to that of Cytoscape.ipynb

In [1]:
""" DEBUG """
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug

/root/Documents


## Helper: <font color="magenta">JupyterCell</font>

<p></p>
 <font color="magenta">JupyterCell</font> is a useful class that allows you to execute a specified *jupyter cell* from the command <font color="magenta">JupyterCell</font><font color="blue">.execute()</font>. This is used to change which <font color="green">widget</font> is displayed inside of the visualizer's **editor cell**.

The editor box is the thing next to the visualizer where a user can change the graph. Notice that the contents of this box changes depending on what the user clicks on the control panel. The Control Panel/ Editor Box functionality is actually a little tricky to create in jupyter and require some special things to be built.


In [ ]:
class JupyterCell():
 
    def __init__(self, cell_text, js_var, text_before="# ", ):
        self.cell_text= cell_text
        self.text_before= text_before
        self.js_var = js_var
        self.search()
    
    def search(self):
        JS("""
        var cell_text= "";
        """+self.js_var+"""= 0;  
        while( ! cell_text.includes('"""+self.text_before+self.cell_text+"""')  ){
            cell_text = Jupyter.notebook.get_cell("""+self.js_var+""").get_text();  //.element["0"].innerText #<-- Old Way
            """+self.js_var+"""++
        }
        """+self.js_var+"""--
        """)
    
    def execute(self):  #<-- This gets the Cell by using the cell text. The cell text must be unique to that cell
        self.search()
        JS("""IPython.notebook.execute_cells(["""+self.js_var+"""])""")

## Helper:  <font color="blue">asyncSleep</font> 

The helper function <font color="blue">asyncSleep</font> allows for a cell to wait while style allowing other python and javascripts to run in the background. The primary use of this is to give enought time for <font color="orange">javascript code</font> to run in the background, before preceding to the next cell.

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

aio_pool = ThreadPoolExecutor(1)
aio_loop = asyncio.new_event_loop()
aio_pool.submit(asyncio.set_event_loop, aio_loop).result()

async def mycoro():
    await asyncio.sleep(.1)
    return 

def asyncSleep():
    aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## Create <font color="blue">Python</font> <font color="magenta">Graph</font> object

Here we create the initial <font color="magenta">Graph</font> class. This class is used to creat and manage the networks visualization. This section only consists of the initialization <font color="magenta">Graph</font> and some of it's basic functions. Most of <font color="magenta">Graph</font> functionality is actually implemented inside of the **Add Widgets** section

The <font color="magenta">Graph</font> Class is what is used to create and manage the networks visualization

In [ ]:
class Graph():
        pass

## Create <font color="olive">HTML</font> Object *"CY"*

This function uses the <font color="magenta">JupyterCell</font> class to find *"My Canvas Box"* and then creates the **cy** html object

In [ ]:
Graph.canvas_box = JupyterCell("My Canvas Box", "canvas_num")
Graph.canvas_box.execute()
asyncSleep()

## <font color="orange">Start Javascript</font>

create the editable the canvas that displays the network

initialize some settings for 'cy' 

In [ ]:
### For Changing highlight features:
#def selectColor(self):

#checkit = cy.$(':selected')
#checkit.style("overlay-color","gray")
#checkit.style("overlay-opacity",".3")
#checkit.style("overlay-padding","8px")

Initialize the javascript object cy, which handles all the manipulations of the html canvas (also called 'cy')

In [ ]:
JS("""
var cy;
require(['cytoscape'], function(cytoscape){ 
    cy = cytoscape({ container: document.getElementById('cy') });
    

cy.io = {}       // <-- where we store all the parameters/variables we create        
cy.maxZoom(14)   // <-- Determine how much window zooms in and out
cy.minZoom(.4)

cy.on("select", 'node, edge' , function(){     //<-- highlight node when selected
    this.style( 'overlay-opacity', .4 );
});
cy.on("unselect", 'node, edge' , function(){   //<-- unhighlight when not selected
    this.style( 'overlay-opacity', 0 );
});

// Create default style for nodes and edges
cy.io.style = {}
cy.io.style.node = { 'overlay-color': 'gray', 'overlay-padding': '2px', 'background-color': 'gray', 'text-background-color':'white',
                        'text-border-width':'2px', 'text-background-padding':'4px' }
cy.io.style.edge = { 'overlay-color': 'gray', 'overlay-padding': '8px', 'line-color': 'gray', 'curve-style': 'bezier', 
                        'text-background-color':'white', 'text-border-width':'2px', 'text-background-padding':'4px' }

// When selecting/unselecting elements, Save selection search (for computational efficiency)                      
cy.on('select unselect', 'node, edge', function(event){ cy.io.selection= cy.$(':selected')  }); 

});
""")
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

# <font color="orange">Javascript Functions</font>

## <font color="orange">Set Styles</font>

<font color="orange">cy.io.style.set</font> sets the style of the node/edge when it's added to the graph. If the parameter, <font color="blue">self</font> is true (default false) then function works the other way: 
>  <font color="orange">cy.io.style</font> $\leftarrow$ element.style() 

In [ ]:
display(Javascript("""
cy.io.style.set = function(ele, self=false){
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;
    
    if( self == false ){ //<-- set properties of element
        for (var property in style ) {
            if(  !/[A-Z]/.test(property) && !property.includes("pie") && ele.style(property) != "NaNpx"  ){   //  remove cytoscape.js errors
                ele.style( property, style[property] )
           }
        }    
    }else{    
        for( var property in ele.style() ){ 
            if(  !/[A-Z]/.test(property) && !property.includes("pie") && ele.style(property) != "NaNpx"  ){   //  remove cytoscape.js errors
                style[ property ] = ele.style( property ) 
           }
        }
    }
};
"""))
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## <font color="orange">addNode</font>

<font color="orange">cy.io.addNode</font>  adds new nodes to the network when the clicks and empty part of the graph.

In [ ]:
display(Javascript("""
cy.io.addNode = function(event){
  if( event.target === cy ){                                                          // <-- If we tapped on the background
      var ele = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      cy.io.style.set( ele )
  }; 
};
cy.on('tap', cy.io.addNode );
"""))
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## <font color="orange">addEdge</font>

<font color="orange">cy.io.addEdge</font> adds a new edge when the use double clicks one nodes and then double clicks another node. <font color="orange">addEdge</font> check for the double clicking and adds a new edge if necessary.

In [ ]:
display(Javascript("""
cy.io.click_hist = ["1", "2", "3", "4"];

cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
    cy.io.click_hist.shift();                                         //<-- Remove oldest click

    if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
      cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

      if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){           //<-- If 2 double clicks
          var ele= cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);    //<-- add edge
          cy.io.style.set( ele )                                                                            //<-- add style of new edge
      };

    }else{                                                 //<-- if we didn't click a node
      cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
    }; 
};
cy.on("click" , cy.io.addEdge );
"""))
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

## <font color="orange">Change Style for Element</font>

<font color="orange">cy.io.style.change</font> is used to edit the style elements.

This process is a little involved because the style variables <font color="orange">cy.io.style.node</font> and <font color="orange">cy.io.style.edge</font> must also be set to the style of the current element, and then changed.

In [ ]:
display(Javascript("""
cy.io.style.change = function(ele, property, value ){
    cy.io.style.set( ele, self=true )                                     // Make style same as node type
    var style = (ele.isNode() ) ? cy.io.style.node : cy.io.style.edge ;   // get style type
    
    typeof(property)
    if( typeof(property)=="string" ){                                     // handle changing a single style property
        style[ property ] = value                                             // change style value
        ele.style( property, value )                                          // change element's value
    }else{                                                                // Multiple properties
        for(var i=0; i<property.length; i++){
            style[ property[i] ] = value[i]
            ele.style( property[i], value[i] )  
        }
    }
};
"""))
aio_pool.submit(aio_loop.run_until_complete, mycoro()).result()

# <font color="green">Widgets</font> 

The purpose of the <font color="green">widgets</font>  section is to create all the basic "tools" that are used to manipulate the cytoscape canvas. We create the tools and the functions that allow them to do their job. Later, in the **"Display Boxes"** section, we combine to the tools together to build a pretty layout

In [ ]:
Graph.w = type("widget_list",(),{})()
Graph.w.time = 0 #<-- used to measure time between update events (see size for example)
Graph.w.vars = {}#<-- used to hold variable classes that a user creates

## <font color="DarkKhaki">Node</font> and <font color="purple">Edge</font> <font color="green">Widgets</font> 

### <font color="green">Color </font> 

In [ ]:
Graph.w.color = ipw.ColorPicker(  )
Graph.w.color

In [ ]:
def color(b):
    values = "['"+Graph.w.color.value+"','"+Graph.w.color.value+"']"
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ["background-color","line-color"], """+values+"""  )
    }
    """))          
Graph.w.color.observe( color )

### <font color="green">Size</font> 

In [ ]:
Graph.w.size = ipw.IntSlider(value=30, min=1, max=200, continuous_update=True )
Graph.w.size 

In [ ]:
def size(b):
    values = "['"+str(Graph.w.size.value)+"','"+str(Graph.w.size.value)+"']" #<-- values for changing node's style
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            cy.io.style.change( ele, ["width","height"], """+values+"""   )   
        }else{
            cy.io.style.change( ele, "width", '"""+str(Graph.w.size.value/6)+"""'   )   
        }
    }
    """))           
Graph.w.size.observe( size )

### <font color="green">Opacity</font>  

In [ ]:
Graph.w.opacity = ipw.FloatSlider(value=.5, min=0, max=1, step=.05, continuous_update=True )
Graph.w.opacity 

In [ ]:
def opacity(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[ i ]
        cy.io.style.change( ele, "opacity", '"""+str(Graph.w.opacity.value)+"""'  )
    }
    """))           
Graph.w.opacity.observe( opacity )

### <font color="green">Z-Index Labels Behind/In-Front</font> <font color="red">Glitchy</font>

In [ ]:
from IPython.core.display import Javascript

In [ ]:
Graph.w.z_index = ipw.IntSlider(value=0, min=0, max=5, continuous_update=True )
Graph.w.z_index

In [ ]:
def z_index(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[ i ]
        cy.io.style.change( ele, "z-index", '"""+str(Graph.w.z_index.value)+"""'  )
    }
    """))               
Graph.w.z_index.observe( z_index )

## <font color="green">General Widgets</font> 

### <font color="green">Undo Delete </font> 

In [ ]:
Graph.w.undo = ipw.Button(description='Undo',button_style='primary', disabled= True )
Graph.w.undo

In [ ]:
def undo(b):
    display(Javascript("cy.add( cy.io.last_deleted );"))   #<-- Add last deleted element
    b.disabled = True
Graph.w.undo.on_click( undo )

### <font color="green">Delete</font> 

In [ ]:
Graph.w.delete = ipw.Button(description='Delete',button_style='danger')
Graph.w.delete

In [ ]:
def delete(b):
    display(Javascript("cy.io.last_deleted = cy.io.selection.remove();"))   #<-- delete elements
    Graph.w.undo.disabled = False
Graph.w.delete.on_click( delete )

### <font color="green">Background Color </font> 

In [ ]:
Graph.w.background = ipw.Button( button_style='danger', icon='image', layout= ipw.Layout(width='auto') )
Graph.w.background

In [ ]:
def background(b):
    display(Javascript(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.color.value}'"))  
Graph.w.background.on_click( background )

### <font color="green">Search</font> 

In [ ]:
Graph.w.search = widgets.Text( description='Search :', continuous_update=False )
Graph.w.search

In [ ]:
def search(b):
    display(Javascript(f"""
    cy.io.selection.unselect()
    cy.$('{Graph.w.search.value}').select()
    """))   
Graph.w.search.observe( search )

### <font color="green">Search Select</font>

In [ ]:
Graph.w.searchSelect = ipw.ToggleButton( icon='eye', layout= ipw.Layout(width='auto')  )
Graph.w.searchSelect

In [ ]:
def searchSelect(b):
    if Graph.w.searchSelect.value:   #<-- when activated
        Graph.w.searchSelect.icon = 'eye-slash'
        display(Javascript("""
        cy.io.to_hide = cy.elements().difference( cy.io.selection )
        cy.io.to_hide.style("visibility", "hidden")
        """))
    else: #<-- when not clicked
        Graph.w.searchSelect.icon = 'eye'
        display(Javascript("""cy.io.to_hide.style("visibility", "visible")"""))
Graph.w.searchSelect.observe( searchSelect )

##  <font color="salmon">Label</font> <font color="green">Widgets</font> 

### <font color="salmon">Label Text</font>

In [ ]:
Graph.w.label = widgets.Text( continuous_update=True )
Graph.w.label

In [ ]:
def label(b):
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, "label", '"""+Graph.w.label.value+"""'  )
    }
    """))           
Graph.w.label.observe( label )

### <font color="salmon">Label Color</font>

In [ ]:
Graph.w.label_color = ipw.ColorPicker( )
Graph.w.label_color

In [ ]:
def labelColor(b):
    js_var = "['"+Graph.w.label_color.value+"','"+Graph.w.label_color.value+"']"
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ['color','text-border-color'], """+js_var+""" )
    }
    """))          
Graph.w.label_color.observe( labelColor )

### <font color="salmon">Label Size</font>

In [ ]:
Graph.w.label_size = ipw.IntSlider(value=16, min=6, max=60)
Graph.w.label_size

In [ ]:
def labelSize(b):
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, 'font-size', '"""+str(Graph.w.label_size.value)+"""px' )
    }
    """))          
Graph.w.label_size.observe( labelSize )

### <font color="salmon">Box</font>  

In [ ]:
Graph.w.label_box = ipw.ToggleButton( button_style='primary', description="Box", layout= ipw.Layout(width='auto') )
Graph.w.label_box

In [ ]:
def labelBox(b):
    js_var = "['1','1']" if Graph.w.label_box.value else "['0','0']"
    display(Javascript("""
    for( var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        cy.io.style.change( ele, ['text-background-opacity','text-border-opacity'], """+js_var+""" )
    }
    """))
Graph.w.label_box.observe( labelBox )

### <font color="DarkKhaki">Node</font> <font color="salmon">Label Position</font>  

In [ ]:
Graph.w.label_pos = ipw.Dropdown( options=['center','top','bottom','left','right'] )
Graph.w.label_pos

In [ ]:
def labelPos(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        var label_pos = '"""+Graph.w.label_pos.value+"""'
        if( ele.isNode() ){
            if( label_pos == 'center'  ){
                cy.io.style.change( ele, ['text-halign','text-valign'], ['center','center'] )
            }else if( label_pos=='top' || label_pos=='bottom'){
                cy.io.style.change( ele, ['text-halign','text-valign'], ['center', label_pos] )                    
            }else{
                cy.io.style.change( ele, ['text-halign','text-valign'], [ label_pos, 'center'] )                
            }
        } 
    }
    """))           
Graph.w.label_pos.observe( labelPos )

## <font color="DarkKhaki">Node Shape</font> 

In [ ]:
Graph.w.shape = ipw.Dropdown( options=['ellipse','triangle','rectangle','rhomboid','diamond','pentagon','hexagon',
                                       'concavehexagon','heptagon','octagon','star','tag','vee'], continuous_update=True )
#Graph.w.shape

In [ ]:
def shape(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            ele.style( "shape", '"""+Graph.w.shape.value+"""' )
        } 
    }
    """))           
Graph.w.shape.observe( shape )

## <font color="purple">Edge</font> <font color="green">Widgets</font> 

### <font color="purple">Edge Arrow Start</font> 

In [ ]:
Graph.w.start = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.start

In [ ]:
def edgeStart(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "source-arrow-color", ele.style("line-color")  )
            ele.style( "source-arrow-shape", '"""+Graph.w.start.value+"""' )
        } 
    }
    """))           
Graph.w.start.observe( edgeStart )

### <font color="purple">Edge Arrow End</font> 

In [ ]:
Graph.w.end = ipw.Dropdown( options=['none','triangle','tee','diamond','square','circle','vee','triangle-tee'] )
#Graph.w.end

In [ ]:
def edgeEnd(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "target-arrow-color", ele.style("line-color")  )
            ele.style( "target-arrow-shape", '"""+Graph.w.end.value+"""' )
        } 
    }
    """))           
Graph.w.end.observe( edgeEnd )

### <font color="purple">Edge Line Style</font> 

In [ ]:
Graph.w.lineStyle = ipw.Dropdown( options=['solid','dotted','dashed'] )
#Graph.w.lineStyle

In [ ]:
def lineStyle(b):
    display(Javascript("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isEdge() ){
            ele.style( "lineStyle", '"""+Graph.w.lineStyle.value+"""' )
        } 
    }
    """))           
Graph.w.lineStyle.observe( lineStyle )

## <font color="hotpink">Variables</font>  <font color="green">Widgets</font> 

**<font color="red">IDEA :</font>** To "hide" widgets just change width's and heights, after changing visibility

In [ ]:
Graph.w.vars = {}

### <font color="hotpink">Change ID</font>  <font color="red">Not Implemented</font>

In [ ]:
Graph.w.id = widgets.Text( description='ID :', continous_update = True  )#, continous_update = False )
Graph.w.id

In [ ]:
def setId(b):
    JS("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
    }
    """)           
Graph.wf.setId = setId
Graph.w.id.observe( Graph.wf.setId )

### <font color="hotpink">Create New  Variable</font> 

In [ ]:
Graph.w.var_create_type= ipw.Select( options=['Number', 'Text'], value='Number', layout=ipw.Layout(width='10%',height='40px',min_width='50px') )
Graph.w.var_create_text= ipw.Text(placeholder='Variable Name:')
Graph.w.var_create_btn = ipw.Button( button_style='primary', description="Create", layout= ipw.Layout(width='auto') )

Graph.w.var_create = ipw.HBox([ Graph.w.var_create_type, Graph.w.var_create_text, Graph.w.var_create_btn ])
Graph.w.var_create.layout.align_items = 'center'
Graph.w.var_create

In [ ]:
def createVar(b):
    if Graph.w.var_create_text.value not in Graph.w.vars.keys() and  Graph.w.var_create_text.value!='' :
        Graph.w.vars[ Graph.w.var_create_text.value ] = Graph.w.var_create_type.value
    
    boxVars = [ x.children[1].label for x in list(Graph.w.varBox.children) ]
    Graph.w.var_add_name.options = list(set(Graph.w.vars.keys()) - set(boxVars))
Graph.w.var_create_btn.on_click( createVar )

### <font color="hotpink">Show Variables</font> 

In [ ]:
Graph.w.varBox = ipw.VBox([])
Graph.w.varBox

In [ ]:
#print(Graph.w.vars)
#Graph.w.varBox.vars = [] #<-- where we store list of varBox's labels

We Create a way to add variables to the HBox

In [ ]:
def add( label, value=None ):
    
    # We create the row (HBox) that is added to the box of variables
    var_label = ipw.HTML("<font size=3 style='justify-content: center;'>"+label+" : </font>") 
    var_value = ipw.Text(continuous_update=True) if Graph.w.vars[label]=='Text' else ipw.FloatText(continuous_update=True)
    if value != None :
        var_value.value = value 
    var_btn   = ipw.Button( button_style='danger', description="Remove", layout= ipw.Layout(width='auto') )
    var = ipw.HBox([ var_label, var_value, var_btn ])
            
    # now we setup remove button
    var_btn.label = label
    def remove(b):
        # Remove variable object from  varBox
        child_idx = [ x.children[1].label for x in list(Graph.w.varBox.children) ].index(b.label)  #<- index of child
        #Graph.w.varBox.vars.pop( child_idx )    #<-- remove label from varBox.vars
        childs = list(Graph.w.varBox.children)
        childs.pop( child_idx )
        Graph.w.varBox.children = tuple(childs)
        # remove data object from selected elments
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.removeData('"""+b.label+"""')
        }
        """))  
    var_btn.on_click( remove )
    
    # add a Variable Updating Function
    var_value.label = label
    def update(change):
        val =  "'"+change['new']+"'" if type(change['new'])== type('') else str(change['new'])
        display(Javascript("""
        for( var i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            ele.data('"""+change['owner'].label+"""', """+val+""" )
        }
        """))
    var_value.observe( update, names='value')        
        
    # Add the row to the varBox
    Graph.w.varBox.children = Graph.w.varBox.children + (var,)
    
Graph.w.varBox.add = add  

### <font color="hotpink">Retreive Variables (from JS)</font> <font color="red">DO Multiple</font>

In [ ]:
#JS("cy.removeListener('select unselect', 'node, edge', cy.io.getVars )")

In [ ]:
JS("""
cy.io.getVars = function(event){
    var selection = cy.$(':selected')
    py.execute("Graph.w.varBox.children = ()")
    if( selection.length == 0 ){ return; }
    
    if( selection.length == 1 ){        
        var data = selection[0].data()
        var data_vars = Object.keys(data).slice(1,)
        for( i in data){            
            if( data[i] != undefined && i!='id' ){
                var val_js = ( typeof(data[i])=="number" ) ? String(data[i]) : '"'+data[i]+'"'
                py.execute('Graph.w.varBox.add("'+i+'",'+val_js+')')
            }
        }
    }else{
        console.log("Inside length MANY")
    
        varsShow = []
        for( var i=0; i < selection.length; i++ ){
            var ele = selection[i]
            var data = ele.data()
            for( key in data ){
                if( key!='id' && key!='ID' && !varsShow.includes(key) && !data[key]==undefined ){
                    varsShow.push( key )
                }
            }
        }
        for( show in varsShow ){
            py.execute('Graph.w.varBox.add("'+show+'")')
        }
    }
};
cy.on('select unselect', 'node, edge', cy.io.getVars )
""")
asyncSleep()

### <font color="hotpink">Add Variable</font> 

In [ ]:
Graph.w.var_add_start = ipw.ToggleButton( button_style='success', icon="plus", layout= ipw.Layout(width='auto') )
Graph.w.var_add_name = ipw.Dropdown(layout= ipw.Layout(width='10%') )
#Graph.w.var_add_name.options = list(set(Graph.w.vars.keys())-set(Graph.w.varBox.vars)) #<-- set intitial options
Graph.w.var_add_text = ipw.Text()
Graph.w.var_add_num  = ipw.FloatText()
Graph.w.var_add_btn = ipw.Button( button_style='primary', description="Add", layout= ipw.Layout(width='auto') )
Graph.w.var_add_error = ipw.HTML("<h4 class='text-danger'>Empty Text</h4>", layout= ipw.Layout(visibility='hidden') )

Graph.w.var_add = ipw.HBox([ Graph.w.var_add_start, Graph.w.var_add_name, Graph.w.var_add_text, Graph.w.var_add_num,
                             Graph.w.var_add_btn, Graph.w.var_add_error  ])
Graph.w.var_add.layout.align_items = 'center'

for w in Graph.w.var_add.children[1:-1] :  #<-- hides all boxes except "start button" and "error box" (error box already hidden)
    w.layout.display = "none" # <--  Other choice is: None
Graph.w.var_add

In [ ]:
def setVarName(b):
    # TO DO: var_add_name should only display variables that are not already set
    if Graph.w.vars[Graph.w.var_add_name.label]  == 'Number':
        Graph.w.var_add_text.layout.display = "none"
        Graph.w.var_add_num.layout.display = None
    else:
        Graph.w.var_add_text.layout.display = None
        Graph.w.var_add_num.layout.display = "none"
        
    Graph.w.var_add_btn.layout.display = None
    
Graph.w.var_add_name.observe( setVarName, names="label" )

In [ ]:
def addVarStart(b):
    if Graph.w.var_add_start.value :  #<-- when activated
        Graph.w.var_add_start.icon = "minus"
        Graph.w.var_add_start.button_style = "danger"
        Graph.w.var_add_name.layout.display = None
        Graph.wf.setVarName("dummy")
    else:                             #<-- when deactivated
        Graph.w.var_add_start.icon = "plus"
        Graph.w.var_add_start.button_style = "success"
        for w in reversed(Graph.w.var_add.children[1:-1]) :  #<-- hides elements, used "reversed" because it looks better when hiding
            w.layout.display = "none"
        # Reset Boxes
        Graph.w.var_add_text.value=''; Graph.w.var_add_num.value=0; Graph.w.var_add_error.layout.visibility='hidden'
        
Graph.w.var_add_start.observe( addVarStart )

In [ ]:
def addVar(b):
    
    if Graph.w.var_add_text.value=='' and Graph.w.vars[Graph.w.var_add_name.label]=='Text' : #<-- if user didn't write anything, stop function
        Graph.w.var_add_error.layout.visibility = 'visible'
        return  
    val = Graph.w.var_add_text.value if Graph.w.vars[Graph.w.var_add_name.label]=='Text' else Graph.w.var_add_num.value
    val_js = "'"+val+"'" if Graph.w.vars[Graph.w.var_add_name.value]=='Text' else str(val)

    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.data( '"""+Graph.w.var_add_name.label+"""', """+val_js+""" )
    }
    """))
    
    Graph.w.varBox.add( Graph.w.var_add_name.label, val )
    
    # Reset Boxes
    Graph.w.var_add_start.value= False
    
Graph.w.var_add_btn.on_click( addVar )

In [ ]:
def optionsUpdate(change):
    boxVars = [ x.children[1].label for x in list(Graph.w.varBox.children) ]
    Graph.w.var_add_name.options = list(set(Graph.w.vars.keys()) - set( boxVars ) )
    Graph.w.var_add_start.value= False
Graph.w.varBox.observe( optionsUpdate  )

In [ ]:
#[x.label for x in list(Graph.w.varBox.children) ]

In [ ]:
#list(Graph.w.varBox.children)

### Before Change

In [ ]:
#Graph.w.var_add = ipw.Dropdown( options=list(set(Graph.w.vars.keys())-set(Graph.w.varBox.vars)), value=None) #,layout= ipw.Layout(width='10%') )
#Graph.w.var_add

```python
def addVar(change):
    Graph.w.varBox.add( Graph.w.var_add.label )
    val_js = "''" if Graph.w.vars[Graph.w.var_add.label]=='Text' else "0"
    display(Javascript("""
    for(var i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        ele.data( '"""+Graph.w.var_add.label+"""', """+val_js+""" )
    }
    """))    
Graph.w.var_add.observe( addVar, names='label' )
```

We add an obsercer to **varBox** which updates the options of the "add" dropdown depending on what variables are still not used

In [ ]:
#boxVars = [ x.children[1].label for x in list(Graph.w.varBox.children) ]

In [ ]:
#list(set(Graph.w.vars.keys()) - set(Graph.w.varBox.vars))

In [ ]:
#Graph.w.vars

# <font color="orchid">Display Box</font>

Here we take the prviously created <font color="green">widgets</font> and we group them rogether into a pretty box.

In [ ]:
Graph.box = type("box",(),{})()

**Padding : ** \[ top/ right/ bottom/ left \]

## Style Box

We add some style/layout properties to the widgets we will use in the style box

In [ ]:
Graph.box.color = ipw.HBox([ ipw.Label('Color :'), Graph.w.color, 
                            ipw.Label('    '), Graph.w.background ])

In [ ]:
Graph.box.size = ipw.HBox([ipw.Label('Size :'), Graph.w.size ])

In [ ]:
Graph.box.shape = ipw.HBox([ipw.Label('Shape :'), Graph.w.shape ])
Graph.box.shape.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.lineStyle = ipw.HBox([ipw.Label('Line Style :'), Graph.w.lineStyle ])
Graph.box.lineStyle.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.opacity = ipw.HBox([ipw.Label('See-Through :'), Graph.w.opacity ])
Graph.box.opacity.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.edge_start = ipw.HBox([ipw.Label('Edge Start :'), Graph.w.start ])
Graph.box.edge_start.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.edge_end = ipw.HBox([ipw.Label('Edge End :'), Graph.w.end ])
Graph.box.edge_end.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.z_index = ipw.HBox([ipw.Label('Behind/In-Front :'), Graph.w.z_index ])
Graph.box.z_index.layout.padding = '10px 0px 0px 0px'

Now we create the style box

In [ ]:
Graph.box.style = ipw.VBox([Graph.box.size,
                            Graph.box.color,
                            Graph.box.opacity,
                            Graph.box.shape,
                            Graph.box.lineStyle,
                            Graph.box.edge_start,
                            Graph.box.edge_end,
                            Graph.box.z_index,
                           ])
#Graph.box.style

## <font color="salmon">Label Box</font>

We add some style/layout properties to the widgets we will use in the label box

In [ ]:
Graph.box.label_text = ipw.HBox([ipw.Label('Text :'), Graph.w.label ])

In [ ]:
Graph.box.label_color = ipw.HBox([ipw.Label('Color :'), Graph.w.label_color, Graph.w.label_box ])
#Graph.box.label_color.layout.justify_content = "space-between"
#Graph.box.label_color

In [ ]:
Graph.box.label_size = ipw.HBox([ipw.Label('Size :'), Graph.w.label_size ])
Graph.box.label_size.layout.padding = '10px 0px 0px 0px'

In [ ]:
Graph.box.label_pos = ipw.HBox([ipw.Label('Position :'), Graph.w.label_pos ])
Graph.box.label_pos.layout.padding = '10px 0px 0px 0px'

In [ ]:
#Graph.w.label_box

Create the label box

In [ ]:
Graph.box.label = ipw.VBox([Graph.box.label_text,
                            ipw.HTML("<hr style='padding: 2px; margin: 2px'>"),
                            Graph.box.label_color,
                            Graph.box.label_size,
                            Graph.box.label_pos,
                           ])
Graph.box.label

## <font color="hotPink">Variables Box</font>

In [ ]:
Graph.box.vars = ipw.HTML("<h4>Not implemented yet</h4>")

## Combine Style, Label, Variables

In [ ]:
Graph.box.combo = widgets.Accordion(children=[ Graph.box.style ,Graph.box.label, Graph.box.vars])
Graph.box.combo.set_title(0, 'Style')
Graph.box.combo.set_title(1, 'Label')
Graph.box.combo.set_title(2, 'Variables')
#Graph.box.combo

## Top Box

In [ ]:
Graph.box.search = ipw.HBox([ Graph.w.search, Graph.w.searchSelect, ipw.HTML("<p>&nbsp&nbsp&nbsp&nbsp&nbsp</p>") ])
Graph.box.search.layout.justify_content = "center"
Graph.box.search.layout.padding = '10px 0px 0px 0px'
#Graph.box.search

In [ ]:
Graph.box.delete = ipw.HBox([ Graph.w.delete, Graph.w.undo ] )
Graph.box.delete.layout.justify_content = "center"
Graph.box.delete.layout.padding = '10px 0px 0px 0px'
#Graph.box.delete

In [ ]:
Graph.box.top = ipw.VBox([Graph.box.search, 
                          Graph.box.delete ])
#Graph.box.end

## Box of Everything

In [ ]:
Graph.box.all = ipw.VBox([  Graph.box.top, Graph.box.combo ])
#Graph.box.all

## <font color="orange">editSelector</font>

<font color="orange">editSelector</font> decides what widgets are displayed inside of the editor box *("My Edit Box")*. What's displayed depends on if there are 1/many of node/edges/both.

In [ ]:
JS("""
    var check = 0;
    var check_var;
    cy.io.select_type = "nothing"


    cy.io.test = function(event){
        cy.$(':selected') = cy.$(':selected')                 //<-- Get selected elements
        cy.io.last_type = cy.io.select_type

        if( cy.$(':selected').length == 0  ){                 //<-- Do nothing if nothing's selected
            return
        }
        if( cy.$(':selected').length == 1 ){                  //<-- If selection lenght is 1
            //DEBUG py.execute("Graph.w.id.value = '"++"'")  //<- put selectors ID in box

            if( cy.$(':selected').isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                cy.io.select_type = "1 node"
                //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 node')")
                IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box 
            }
            if( cy.$(':selected').isEdge() &&  cy.io.last_type != "1 edge" ){                     //<-- If it's an edge
                cy.io.select_type = "1 edge"
                //DEBUG py.execute("Graph.editor_box.widg = Graph.box.show('1 edge')")
                IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box
            }

        } else {                                           //<-- If Many elements
            console.log("many items")
        }

    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")

## Display Box

In [ ]:

#Graph.editor_box = JupyterCell("My Edit Box", "edit_num")
#Graph.editor_box.widg = ipw.HTML("<h5> </h5>") #<-- widget initially empty
#Graph.editor_box.execute()


# Visualize (Mock Setup)

In [ ]:
# My Edit Box
display( Graph.box.all )



In [ ]:
# My Canvas Box
display(HTML("""<style> .cy { width: 50vw; height: 30vw}</style>   //DEBUG: made width 50vw
                <div id="cy" class="cy"></div>"""))

# Scratch Paper

In [ ]:
# DEBUG: Remove Listeners
JS("""
//cy.removeListener('tap', handler);
//cy.removeListener( "click" ) 
//cy.removeListener( "tap" )
//cy.removeListener( "select" )
//cy.removeListener( "unselect" )
""")

In [ ]:
JS("""
    var check = 0;
    var check_var;

    cy.io.test = function(event){
        cy.$(':selected') = cy.$(':selected')
        
        check += 1
        console.log( String(check) +": selection changed")
        check_var = 
        cehck_var2 = event
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")


In [ ]:
import inspect
lines = inspect.getsource(interactive )
print(lines)

In [ ]:
"""
JS(

cyto.layout({ name: 'circle'}).run();

)
"""

In [ ]:
%%js
Object.keys(saved_style3).forEach(function(key,index) {
    console.log(key,index)
    // key: the name of the object key
    // index: the ordinal position of the key within the object 
});

In [ ]:
#checkit.on_trait_change()

In [ ]:
checkit = ipw.Text("hello",continuous_update=True)
def onChange(b):
    print( change['new'], names='value' )

checkit.observe( onChange )
checkit

In [ ]:
checkit.continuous_update

In [ ]:

int_range = widgets.IntSlider()
display(int_range)

def on_value_change(change):
    print(change )

int_range.observe(on_value_change, names='value')

In [ ]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

In [ ]:
def f(x):
    print(x)
    pass
interactive(f, x=(0.0,10.0));

In [ ]:
w

In [ ]:
w.layout.width = "100%"

In [ ]:
a = ipw.Text()
b = ipw.Label()
def f(x,y):
    print(x.value, y.value)


w = interactive(f, a, b)

chekcing observe

In [ ]:
x = []
checkit = ipw.FloatText(description="label", continuous_update=True)
checkit.myVar="howdy"
def checkitFunc(change):
        global x
        x = change
        print(change['new'], type(change['new']))
checkit.observe(checkitFunc, names='value')


In [ ]:
checkit

In [ ]:
x['new']

In [ ]:
checkit.get_state

In [ ]:
import inspect
lines = inspect.getsource(checkit.add_class)
print(lines)